In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import kagglehub

path = kagglehub.dataset_download("valakhorasani/bank-transaction-dataset-for-fraud-detection")

print("Path to dataset files:", path)

100%|██████████| 102k/102k [00:00<00:00, 35.7MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/valakhorasani/bank-transaction-dataset-for-fraud-detection/versions/4


In [3]:
import os
path_to_csv = os.listdir(path)[0]

In [4]:
df = pd.read_csv(os.path.join(path,path_to_csv))
df.head()

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39


In [5]:
df = df.drop(["TransactionID","AccountID"],axis = 1)
df.columns

Index(['TransactionAmount', 'TransactionDate', 'TransactionType', 'Location',
       'DeviceID', 'IP Address', 'MerchantID', 'Channel', 'CustomerAge',
       'CustomerOccupation', 'TransactionDuration', 'LoginAttempts',
       'AccountBalance', 'PreviousTransactionDate'],
      dtype='object')

In [6]:
df.isna().sum()

,0
TransactionAmount,0
TransactionDate,0
TransactionType,0
Location,0
DeviceID,0
IP Address,0
MerchantID,0
Channel,0
CustomerAge,0
CustomerOccupation,0


In [7]:
df.drop_duplicates(inplace = True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   TransactionAmount        2512 non-null   float64
 1   TransactionDate          2512 non-null   object 
 2   TransactionType          2512 non-null   object 
 3   Location                 2512 non-null   object 
 4   DeviceID                 2512 non-null   object 
 5   IP Address               2512 non-null   object 
 6   MerchantID               2512 non-null   object 
 7   Channel                  2512 non-null   object 
 8   CustomerAge              2512 non-null   int64  
 9   CustomerOccupation       2512 non-null   object 
 10  TransactionDuration      2512 non-null   int64  
 11  LoginAttempts            2512 non-null   int64  
 12  AccountBalance           2512 non-null   float64
 13  PreviousTransactionDate  2512 non-null   object 
dtypes: float64(2), int64(3),

In [9]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'])
df['transaction_hour'] = df['TransactionDate'].dt.hour
df['transaction_dayofweek'] = df['TransactionDate'].dt.dayofweek
df['prev_transaction_hour'] = df['PreviousTransactionDate'].dt.hour
df['prev_transaction_dayofweek'] = df['PreviousTransactionDate'].dt.dayofweek
df = df.drop(["TransactionDate","PreviousTransactionDate"],axis=1)
df.head()

,TransactionAmount,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,transaction_hour,transaction_dayofweek,prev_transaction_hour,prev_transaction_dayofweek
0,14.09,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,16,1,8,0
1,376.24,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,16,1,8,0
2,126.29,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,18,0,8,0
3,184.50,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,16,4,8,0
4,13.45,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,17,0,8,0


In [10]:
import ipaddress
df['ip_address'] = df['IP Address'].apply(lambda x: int(ipaddress.ip_address(x)))
df = df.drop(["IP Address"],axis = 1)
df.head()

,TransactionAmount,TransactionType,Location,DeviceID,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,transaction_hour,transaction_dayofweek,prev_transaction_hour,prev_transaction_dayofweek,ip_address
0,14.09,Debit,San Diego,D000380,M015,ATM,70,Doctor,81,1,5112.21,16,1,8,0,2730941020
1,376.24,Debit,Houston,D000051,M052,ATM,68,Doctor,141,1,13758.91,16,1,8,0,227884292
2,126.29,Debit,Mesa,D000235,M009,Online,19,Student,56,1,1122.35,18,0,8,0,3613495197
3,184.50,Debit,Raleigh,D000187,M002,Online,26,Student,25,1,8569.06,16,4,8,0,3356352918
4,13.45,Credit,Atlanta,D000308,M091,Online,26,Student,198,1,7429.40,17,0,8,0,1101267812


In [11]:
df = df.drop(["MerchantID","DeviceID"],axis = 1)
df.head()

,TransactionAmount,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,transaction_hour,transaction_dayofweek,prev_transaction_hour,prev_transaction_dayofweek,ip_address
0,14.09,Debit,San Diego,ATM,70,Doctor,81,1,5112.21,16,1,8,0,2730941020
1,376.24,Debit,Houston,ATM,68,Doctor,141,1,13758.91,16,1,8,0,227884292
2,126.29,Debit,Mesa,Online,19,Student,56,1,1122.35,18,0,8,0,3613495197
3,184.50,Debit,Raleigh,Online,26,Student,25,1,8569.06,16,4,8,0,3356352918
4,13.45,Credit,Atlanta,Online,26,Student,198,1,7429.40,17,0,8,0,1101267812


In [12]:
df_numerical = df.select_dtypes(include = np.number)
df_categorical = df.select_dtypes(exclude = np.number)
df_numerical.columns,df_categorical.columns

(Index(['TransactionAmount', 'CustomerAge', 'TransactionDuration',
        'LoginAttempts', 'AccountBalance', 'transaction_hour',
        'transaction_dayofweek', 'prev_transaction_hour',
        'prev_transaction_dayofweek', 'ip_address'],
       dtype='object'),
 Index(['TransactionType', 'Location', 'Channel', 'CustomerOccupation'], dtype='object'))

In [13]:
for col in df_categorical.columns:
    print(col,df_categorical[col].unique())

TransactionType ['Debit' 'Credit']
Location ['San Diego' 'Houston' 'Mesa' 'Raleigh' 'Atlanta' 'Oklahoma City'
 'Seattle' 'Indianapolis' 'Detroit' 'Nashville' 'Albuquerque' 'Memphis'
 'Louisville' 'Denver' 'Austin' 'Columbus' 'Los Angeles' 'Las Vegas'
 'Fort Worth' 'Miami' 'Milwaukee' 'Baltimore' 'New York' 'San Francisco'
 'San Jose' 'San Antonio' 'Philadelphia' 'Charlotte' 'Tucson' 'Chicago'
 'Sacramento' 'Kansas City' 'Omaha' 'Virginia Beach' 'Dallas' 'Boston'
 'Jacksonville' 'Phoenix' 'Washington' 'El Paso' 'Colorado Springs'
 'Fresno' 'Portland']
Channel ['ATM' 'Online' 'Branch']
CustomerOccupation ['Doctor' 'Student' 'Retired' 'Engineer']


In [14]:
import joblib
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
for col in df_categorical.columns:
    df[col] = encoder.fit_transform(df[[col]])
    joblib.dump(encoder, f'{col}_ordinal_encoder.pkl')
df.head()

,TransactionAmount,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,transaction_hour,transaction_dayofweek,prev_transaction_hour,prev_transaction_dayofweek,ip_address
0,14.09,1.0,36.0,0.0,70,0.0,81,1,5112.21,16,1,8,0,2730941020
1,376.24,1.0,15.0,0.0,68,0.0,141,1,13758.91,16,1,8,0,227884292
2,126.29,1.0,23.0,2.0,19,3.0,56,1,1122.35,18,0,8,0,3613495197
3,184.50,1.0,33.0,2.0,26,3.0,25,1,8569.06,16,4,8,0,3356352918
4,13.45,0.0,1.0,2.0,26,3.0,198,1,7429.40,17,0,8,0,1101267812


In [15]:
from sklearn.ensemble import IsolationForest
import pandas as pd


iso_forest = IsolationForest(
    n_estimators=100,
    contamination=0.02,
    random_state=42
)
iso_forest.fit(df.values)

# -1 = anomaly (fraud), 1 = normal (legit)
df['Predicted_Fraud'] = iso_forest.predict(df.values)
df['Predicted_Fraud'] = df['Predicted_Fraud'].map({1: 0, -1: 1})

print(df['Predicted_Fraud'].value_counts())

Predicted_Fraud
0    2461
1      51
Name: count, dtype: int64


In [16]:
df.head()

,TransactionAmount,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,transaction_hour,transaction_dayofweek,prev_transaction_hour,prev_transaction_dayofweek,ip_address,Predicted_Fraud
0,14.09,1.0,36.0,0.0,70,0.0,81,1,5112.21,16,1,8,0,2730941020,0
1,376.24,1.0,15.0,0.0,68,0.0,141,1,13758.91,16,1,8,0,227884292,0
2,126.29,1.0,23.0,2.0,19,3.0,56,1,1122.35,18,0,8,0,3613495197,0
3,184.50,1.0,33.0,2.0,26,3.0,25,1,8569.06,16,4,8,0,3356352918,0
4,13.45,0.0,1.0,2.0,26,3.0,198,1,7429.40,17,0,8,0,1101267812,0


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib

X = df.drop(["Predicted_Fraud"],axis=1)
y = df["Predicted_Fraud"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

joblib.dump(scaler, 'scaler.pkl')

X_test, X_val, y_test, y_val = train_test_split(X_test_scaled, y_test, test_size = 0.5)
print(X_train_scaled.shape,X_test_scaled.shape,X_val.shape)

(1758, 14) (754, 14) (377, 14)


In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [19]:
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32).to(device)
    self.y = torch.tensor(y.values,dtype=torch.float32).to(device)
  def __len__(self):
    return len(self.x)
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
train_data = dataset(X_train_scaled, y_train)
val_data = dataset(X_val, y_val)
test_data = dataset(X_test, y_test)

In [20]:
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 1e-4

In [21]:
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle= True)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle= True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle= True)

In [22]:
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.l1 = nn.Linear(X.shape[1],10)
    self.l2 = nn.Linear(10,1)
    self.out = nn.Sigmoid()
  def forward(self,x):
    x = self.l1(x)
    x = self.l2(x)
    x = self.out(x)
    return x

In [23]:
!pip install torchsummary

In [24]:
from torchsummary import summary
model = Model().to(device)
summary(model, input_size=(X.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             150
            Linear-2                    [-1, 1]              11
           Sigmoid-3                    [-1, 1]               0
Total params: 161
Trainable params: 161
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [25]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [26]:
for epoch in range(EPOCHS):
  train_loss = 0
  train_acc = 0
  val_acc = 0
  val_loss = 0
  for inputs,labels in train_dataloader:
    optimizer.zero_grad()
    prediction = model(inputs)
    loss = criterion(prediction,labels.unsqueeze(1))
    acc = (prediction.round() == labels.unsqueeze(1)).sum().item() / len(labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
    train_acc += acc
  with torch.no_grad():
    for inputs,labels in val_dataloader:
      prediction = model(inputs)
      loss = criterion(prediction,labels.unsqueeze(1))
      acc = (prediction.round() == labels.unsqueeze(1)).sum().item() / len(labels)
      val_loss += loss.item()
      val_acc += acc
  print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss/len(train_dataloader):.3f} | Train Acc: {train_acc/len(train_dataloader):.3f} | Val Loss: {val_loss/len(val_dataloader):.3f} | Val Acc: {val_acc/len(val_dataloader):.3f}")

Epoch 1/100 | Train Loss: 0.533 | Train Acc: 0.978 | Val Loss: 0.517 | Val Acc: 0.986
Epoch 2/100 | Train Loss: 0.507 | Train Acc: 0.978 | Val Loss: 0.490 | Val Acc: 0.987
Epoch 3/100 | Train Loss: 0.482 | Train Acc: 0.978 | Val Loss: 0.464 | Val Acc: 0.987
Epoch 4/100 | Train Loss: 0.458 | Train Acc: 0.978 | Val Loss: 0.439 | Val Acc: 0.987
Epoch 5/100 | Train Loss: 0.434 | Train Acc: 0.978 | Val Loss: 0.416 | Val Acc: 0.986
Epoch 6/100 | Train Loss: 0.412 | Train Acc: 0.978 | Val Loss: 0.393 | Val Acc: 0.987
Epoch 7/100 | Train Loss: 0.390 | Train Acc: 0.978 | Val Loss: 0.371 | Val Acc: 0.987
Epoch 8/100 | Train Loss: 0.370 | Train Acc: 0.978 | Val Loss: 0.350 | Val Acc: 0.987
Epoch 9/100 | Train Loss: 0.350 | Train Acc: 0.978 | Val Loss: 0.331 | Val Acc: 0.986
Epoch 10/100 | Train Loss: 0.332 | Train Acc: 0.978 | Val Loss: 0.312 | Val Acc: 0.986
Epoch 11/100 | Train Loss: 0.314 | Train Acc: 0.978 | Val Loss: 0.293 | Val Acc: 0.987
Epoch 12/100 | Train Loss: 0.298 | Train Acc: 0.978 

In [27]:
with torch.no_grad():
  test_acc = 0
  for inputs,labels in test_dataloader:
    prediction = model(inputs)
    acc = (prediction.round() == labels.unsqueeze(1)).sum().item() / len(labels)
    test_acc += acc
  print(f"Test Acc: {test_acc/len(test_dataloader)*100:.1f}%")

Test Acc: 98.2%


In [28]:
import pandas as pd
import ipaddress
import joblib
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
model.to(device)

raw_data = {
    "TransactionAmount": [10000.00, 50.00, 500.00],
    "TransactionType": ["Debit", "Credit", "Debit"],
    "Location": ["Nigeria", "USA", "Mumbai"],
    "Channel": ["online", "mobile", "online"],
    "CustomerAge": [28, 34, 22],
    "CustomerOccupation": ["engineer", "teacher", "student"],
    "TransactionDuration": [5, 120, 300],
    "LoginAttempts": [12, 1, 15],
    "AccountBalance": [200.00, 5000.00, 400.00],
    "TransactionDate": ["2025-09-24 02:15:00", "2025-09-24 14:00:00", "2025-09-24 01:00:00"],
    "PreviousTransactionDate": ["2025-09-23 18:00:00", "2025-09-23 10:00:00", "2025-09-23 23:50:00"],
    "ip_address": ["41.76.12.5", "192.168.0.5", "203.0.113.77"],
}

inf_df = pd.DataFrame(raw_data)

inf_df["TransactionDate"] = pd.to_datetime(inf_df["TransactionDate"])
inf_df["PreviousTransactionDate"] = pd.to_datetime(inf_df["PreviousTransactionDate"])
inf_df["transaction_hour"] = inf_df["TransactionDate"].dt.hour
inf_df["transaction_dayofweek"] = inf_df["TransactionDate"].dt.dayofweek
inf_df["prev_transaction_hour"] = inf_df["PreviousTransactionDate"].dt.hour
inf_df["prev_transaction_dayofweek"] = inf_df["PreviousTransactionDate"].dt.dayofweek

inf_df["ip_address"] = inf_df["ip_address"].apply(lambda x: int(ipaddress.ip_address(x)))

inf_df = inf_df.drop(columns=["TransactionDate","PreviousTransactionDate"])

categorical_cols = ['TransactionType', 'Location', 'Channel', 'CustomerOccupation']
for col in categorical_cols:
    encoder = joblib.load(f'{col}_ordinal_encoder.pkl')
    inf_df[col] = encoder.transform(inf_df[[col]])

inf_df = inf_df[X.columns]

scaler = joblib.load('scaler.pkl')
inf_df_scaled = scaler.transform(inf_df)

inf_tensor = torch.tensor(inf_df_scaled, dtype=torch.float32).to(device)
with torch.no_grad():
    preds = model(inf_tensor)

fraud_labels = ["Fraud" if round(p.item())==1 else "Clean" for p in preds]

print("Predictions:", fraud_labels)


Predictions: ['Fraud', 'Clean', 'Fraud']
